# Managing Your Data Requests with `ecmwf-datastores-client`

This guide shows you how to track and manage all the climate/atmosphere data requests you've submitted to the Copernicus Data Store. Think of this as your "data request history manager."

## What You Need Before Starting

1. An **active internet connection**
2. **Python** installed on your computer
3. The `ecmwf-datastores-client` package installed, otherwise uncomment and run the next cell
4. A **CDS account** or **ADS account** with your API key set up (see the "Getting Started" notebook)

In [ ]:
# !pip install -U ecmwf-datastores-client

In [ ]:
import os

import pandas as pd
from IPython.display import display

from ecmwf.datastores import Client

**Tip:** If you don’t want to see warnings while running your notebook, you can uncomment the following cell:

In [ ]:
# import warnings
# warnings.filterwarnings("ignore")

## Connect to the Data Store

First, let's establish a connection to the data store:

In [ ]:
client = Client()

# Check that we're connected successfully
connection_info = client.check_authentication()
print("✅ Connected successfully to the Data Store!")

## View Your Data Request History

Let's retrieve information about all your past data requests:

In [ ]:
# Get a list of your data requests, newest first
print("Retrieving your data request history...")
jobs = client.get_jobs(sortby="-created")

# Create a list to store all the request IDs and their status
request_ids = []

# Loop through all pages of results (if there are many requests)
while jobs is not None:
    # Add each job's ID and status to our list
    request_ids.extend([job["jobID"], job["status"]] for job in jobs.json["jobs"])
    # Move to the next page of results
    jobs = jobs.next

print(f"Found {len(request_ids)} data requests in your history!")

## Get Detailed Information About Your Requests

Now let's collect more detailed information about each of your data requests:

In [ ]:
print("Gathering detailed information about each request...")
jobs_info = []

# Loop through each request and collect its details
for request_id, request_status in request_ids:
    # Get the full details of this particular request
    remote = client.get_remote(request_id)

    # Get date components
    year = ", ".join(filter(None, remote.request.get("year", [None])))
    month = ", ".join(filter(None, remote.request.get("month", [None])))
    day = ", ".join(filter(None, remote.request.get("day", [None])))

    # Combine date information
    date_info = f"{year}-{month}-{day}" if all([year, month, day]) else "Not specified"

    # Extract and organize the important information
    jobs_info.append(
        {
            "Request ID": request_id,
            "Status": request_status,
            "Dataset": remote.collection_id,
            "Created": remote.created_at.strftime("%Y-%m-%d %H:%M"),
            "Completed": remote.finished_at.strftime("%Y-%m-%d %H:%M")
            if remote.finished_at
            else "Not finished",
            "Variable": ", ".join(filter(None, remote.request.get("variable", [None]))),
            "Date": date_info,
            "Format": remote.request.get("data_format", "Unknown"),
        }
    )

print("✅ Information collected successfully!")

In [ ]:
import time
print(len(request_ids))
for i, request_id in enumerate(request_ids):
    print(i, request_id[0])
    client.get_remote(request_id[0])
    time.sleep(1)

## Create an Interactive Table of Your Requests

Now let's display all your data requests in an easy-to-use interactive table:

In [ ]:
# Create a table from our collected information
df_jobs = pd.DataFrame(jobs_info)

# Render DataFrame as HTML to avoid truncation
display(df_jobs)

In [ ]:
df_jobs["Request ID"][0]

## Working with Your Past Data Requests

Here are some examples of what you can do with your past requests:

### Download Data from a Past Request

In [ ]:
# If you want to download data from a specific request:
request_id = (
    "49b3a61b-7310-4da3-b22f-f9e85f2a9821"  # Replace with an ID from your table
)

# Check if the request was successful
remote = client.get_remote(request_id)
if remote.status == "successful":
    print(
        f"Downloading data from request made on {remote.created_at.strftime('%Y-%m-%d')}..."
    )
    remote.download(target="my_past_request_data.grib")
    print("✅ Download complete!")
else:
    print(f"This request has status '{remote.status}' and cannot be downloaded.")

### Delete an Old Request

In [ ]:
# If you want to remove an old request:
request_id = (
    "49b3a61b-7310-4da3-b22f-f9e85f2a9821"  # Replace with an ID from your table
)

# Delete the request
remote = client.get_remote(request_id)
remote.delete()
print("✅ Request deleted successfully!")

## Tips for Managing Your Data Requests

- **Sort by date**: Keep track of when you made each request
- **Look at the status**: "successful" means the data is ready to download
- **Check file formats**: Make sure you know what format your data is in before downloading
- **Keep track of request IDs**: Save important request IDs for data you might need again
- **Clean up periodically**: Delete old requests you no longer need
